In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install -q transformers datasets accelerate sentencepiece nltk scikit-learn torch gensim rouge-score seqeval

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 70.1 MB/s eta 0:00:00


Step 2 : Install Libraries

In [ ]:
import os, re, numpy as np, pandas as pd, nltk
nltk.download('punkt')
nltk.download('stopwords')

import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Input, Bidirectional, TimeDistributed
import joblib



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Step 3 – Load & Clean Dataset

In [ ]:
# Load TSV dataset
data = pd.read_csv(
    '/content/drive/MyDrive/PENS/news.tsv',
    sep='\t',
    engine='python',
    quoting=3,
    on_bad_lines='skip'
)

# Keep relevant columns
data = data.rename(columns={'News body':'text','Category':'category','Entity content':'entities'})
data = data[['text','category','entities']]
data = data.dropna(subset=['text','category'])
data['entities'] = data['entities'].fillna('')

# Clean text
def clean_text(text):
    text = re.sub(r'<.*?>','',text)
    text = re.sub(r'http\S+','',text)
    text = re.sub(r'[^a-zA-Z\s]','',text)
    text = text.lower()
    return text.strip()

data['clean_text'] = data['text'].astype(str).apply(clean_text)

# Keep categories with >=2 samples
valid_categories = data['category'].value_counts()[lambda x: x>=2].index
data = data[data['category'].isin(valid_categories)]


Step 4 – Label Encoding & Train-Test Split

In [ ]:
label_encoder = LabelEncoder()
data['label'] = label_encoder.fit_transform(data['category'])

X_train, X_test, y_train, y_test = train_test_split(
    data['clean_text'],
    data['label'],
    test_size=0.2,
    random_state=42,
    stratify=data['label']
)


Step 5 – ML Classification (TF-IDF + Logistic Regression)

In [ ]:
tfidf = TfidfVectorizer(max_features=5000, ngram_range=(1,2), stop_words='english')
X_train_vec = tfidf.fit_transform(X_train)
X_test_vec = tfidf.transform(X_test)

model = LogisticRegression(max_iter=1000)
model.fit(X_train_vec, y_train)

y_pred = model.predict(X_test_vec)
print(classification_report(label_encoder.inverse_transform(y_test),
                            label_encoder.inverse_transform(y_pred)))


               precision    recall  f1-score   support

        autos       0.88      0.80      0.83      1086
entertainment       0.69      0.40      0.51       280
      finance       0.84      0.77      0.80      2052
 foodanddrink       0.84      0.79      0.81      1048
       health       0.83      0.74      0.78       750
         kids       0.00      0.00      0.00        59
    lifestyle       0.64      0.65      0.64      1437
       movies       0.77      0.57      0.66       365
        music       0.74      0.58      0.65       485
         news       0.69      0.81      0.74      5081
       sports       0.93      0.95      0.94      5953
       travel       0.67      0.53      0.59      1055
           tv       0.63      0.59      0.61       751
        video       0.40      0.44      0.42       917
      weather       0.76      0.65      0.70       657

     accuracy                           0.77     21976
    macro avg       0.69      0.62      0.65     21976
 weighte

In [ ]:
joblib.dump(tfidf, '/content/drive/MyDrive/PENS/tfidf.pkl')
joblib.dump(model, '/content/drive/MyDrive/PENS/logreg_model.pkl')
joblib.dump(label_encoder, '/content/drive/MyDrive/PENS/label_encoder.pkl')

['/content/drive/MyDrive/PENS/label_encoder.pkl']

STEP 5 – DL Tokenization + Padding

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

MAX_WORDS = 20000
MAX_LEN = 200

tokenizer_dl = Tokenizer(
    num_words=MAX_WORDS,
    oov_token="<OOV>"
)

tokenizer_dl.fit_on_texts(data['clean_text'])


In [ ]:
sequences = tokenizer_dl.texts_to_sequences(
    data['clean_text']
)


In [ ]:
padded_sequences = pad_sequences(
    sequences,
    maxlen=MAX_LEN,
    padding='post',
    truncating='post'
)


Step 6 – DL Classification (Word2Vec + LSTM)

In [ ]:
from sklearn.model_selection import train_test_split

X_train_dl, X_test_dl, y_train_dl, y_test_dl = train_test_split(
    padded_sequences,
    data['label'],
    test_size=0.2,
    random_state=42,
    stratify=data['label']
)


In [ ]:
from gensim.models import Word2Vec

sentences = [text.split() for text in data['clean_text']]

w2v_model = Word2Vec(
    sentences=sentences,
    vector_size=100,   # keep small to avoid RAM issues
    window=5,
    min_count=2,
    workers=4
)


In [ ]:
MAX_WORDS = 20000
MAX_LEN = 200
EMBEDDING_DIM = 100   # MUST match vector_size above


In [ ]:
import numpy as np

word_index = tokenizer_dl.word_index

embedding_matrix = np.zeros((MAX_WORDS, EMBEDDING_DIM))

for word, i in word_index.items():
    if i < MAX_WORDS and word in w2v_model.wv:
        embedding_matrix[i] = w2v_model.wv[word]


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

model_dl = Sequential([
    Embedding(
        input_dim=MAX_WORDS,
        output_dim=EMBEDDING_DIM,
        weights=[embedding_matrix],
        input_length=MAX_LEN,
        trainable=False
    ),
    LSTM(64),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dense(len(np.unique(data['label'])), activation='softmax')
])


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
model_dl.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

model_dl.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │     2,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,000,000 (7.63 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 2,000,000 (7.63 MB)

In [ ]:
history = model_dl.fit(
    X_train_dl,
    y_train_dl,
    epochs=5,
    batch_size=16,     # reduce RAM usage
    validation_split=0.1
)


Epoch 1/5
4945/4945 ━━━━━━━━━━━━━━━━━━━━ 61s 11ms/step - accuracy: 0.5134 - loss: 1.6079 - val_accuracy: 0.6279 - val_loss: 1.1679
Epoch 2/5
4945/4945 ━━━━━━━━━━━━━━━━━━━━ 53s 11ms/step - accuracy: 0.6232 - loss: 1.2179 - val_accuracy: 0.6684 - val_loss: 1.0366
Epoch 3/5
4945/4945 ━━━━━━━━━━━━━━━━━━━━ 53s 11ms/step - accuracy: 0.6588 - loss: 1.1032 - val_accuracy: 0.6893 - val_loss: 0.9671
Epoch 4/5
4945/4945 ━━━━━━━━━━━━━━━━━━━━ 53s 11ms/step - accuracy: 0.6765 - loss: 1.0292 - val_accuracy: 0.6950 - val_loss: 0.9305
Epoch 5/5
4945/4945 ━━━━━━━━━━━━━━━━━━━━ 55s 11ms/step - accuracy: 0.6873 - loss: 0.9943 - val_accuracy: 0.7068 - val_loss: 0.9070


In [ ]:
from sklearn.metrics import classification_report

y_pred = model_dl.predict(X_test_dl).argmax(axis=1)

print(classification_report(
    y_test_dl,
    y_pred
))


687/687 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step
              precision    recall  f1-score   support

           0       0.82      0.77      0.80      1086
           1       0.25      0.05      0.09       280
           3       0.75      0.76      0.76      2052
           4       0.73      0.82      0.77      1048
           5       0.73      0.73      0.73       750
           6       0.00      0.00      0.00        59
           7       0.55      0.51      0.53      1437
           8       0.35      0.48      0.41       365
           9       0.46      0.35      0.40       485
          10       0.69      0.68      0.69      5081
          11       0.95      0.93      0.94      5953
          12       0.58      0.41      0.48      1055
          13       0.45      0.11      0.17       751
          14       0.31      0.71      0.44       917
          15       0.60      0.75      0.67       657

    accuracy                           0.71     21976
   macro avg       0.55      0.54      

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import joblib

model_dl.save('/content/drive/MyDrive/PENS/lstm_w2v_model.h5')
joblib.dump(
    tokenizer_dl,
    '/content/drive/MyDrive/PENS/dl_tokenizer.pkl'
)

print("DL model and tokenizer saved successfully")


DL model and tokenizer saved successfully


In [ ]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "NO GPU")


True
Tesla T4


Step 7 BERT Classification

In [ ]:
!pip install -q transformers torch
import torch
from transformers import BertTokenizerFast, BertForSequenceClassification

DEVICE = torch.device("cpu")

tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

model_bert = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=len(label_encoder.classes_)
)

model_bert.to(DEVICE)
model_bert.eval()
import torch

def bert_predict(text):
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=64
    )

    inputs = {k: v.to(DEVICE) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model_bert(**inputs)

    pred_label = torch.argmax(outputs.logits, dim=1).item()
    return label_encoder.inverse_transform([pred_label])[0]
sample_text = data['clean_text'].iloc[0]

print("Text:", sample_text[:200])
print("Predicted Category:", bert_predict(sample_text))
def bert_predict_batch(texts, batch_size=8):
    preds = []

    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]

        inputs = tokenizer(
            batch,
            return_tensors="pt",
            truncation=True,
            padding=True,
            max_length=64
        )

        with torch.no_grad():
            outputs = model_bert(**inputs)

        batch_preds = torch.argmax(outputs.logits, dim=1).tolist()
        preds.extend(batch_preds)

    return label_encoder.inverse_transform(preds)
from sklearn.metrics import classification_report

sample_df = data.sample(n=500, random_state=42)

y_true = label_encoder.inverse_transform(sample_df['label'])
y_pred = bert_predict_batch(sample_df['clean_text'].tolist())

print(classification_report(y_true, y_pred))


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Text: only five internationals allowed count em five so first off we should say per our usual atlanta united lineup predictions this will be wrong why will it be wrong well aside from the obvious we still d
Predicted Category: tv
               precision    recall  f1-score   support

        autos       0.00      0.00      0.00        32
entertainment       0.00      0.00      0.00         8
      finance       0.00      0.00      0.00        39
 foodanddrink       0.00      0.00      0.00        20
       health       0.00      0.00      0.00        20
         kids       0.00      0.00      0.00         1
    lifestyle       0.00      0.00      0.00        40
       movies       0.00      0.00      0.00         8
        music       0.00      0.00      0.00         9
         news       0.00      0.00      0.00       118
       sports       0.00      0.00      0.00       139
       travel       0.00      0.00      0.00        20
           tv       0.03      0.65      0.05        17

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Step 8 – NER (BiLSTM)

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
# Prepare BIO tags
# LIMIT DATA to avoid CPU freeze
data_ner = data[['text', 'entities']].dropna().sample(
    n=3000,   # <= VERY IMPORTANT
    random_state=42
)
from nltk.tokenize import word_tokenize

def create_bio_tags(text, entity):
    words = word_tokenize(text.lower())
    tags = ['O'] * len(words)

    if isinstance(entity, str) and entity.strip():
        e_words = word_tokenize(entity.lower())
        e_len = len(e_words)

        for i in range(len(words) - e_len + 1):
            if words[i:i+e_len] == e_words:
                tags[i] = 'B-ENT'
                for j in range(1, e_len):
                    tags[i+j] = 'I-ENT'
                break

    return words, tags
sentences = []
labels = []

for text, ent in zip(data_ner['text'], data_ner['entities']):
    w, t = create_bio_tags(text, ent)
    sentences.append(w)
    labels.append(t)

 # Tokenization & Padding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

word_tokenizer = Tokenizer(oov_token="<OOV>")
word_tokenizer.fit_on_texts(sentences)

X = word_tokenizer.texts_to_sequences(sentences)

MAX_LEN = 100
X_padded = pad_sequences(X, maxlen=MAX_LEN, padding='post', truncating='post')

tag2idx = {'O': 0, 'B-ENT': 1, 'I-ENT': 2}
idx2tag = {v: k for k, v in tag2idx.items()}

y = [[tag2idx[tag] for tag in seq] for seq in labels]
y_padded = pad_sequences(y, maxlen=MAX_LEN, padding='post', truncating='post')



In [ ]:
# Build NER BiLSTM
VOCAB_SIZE = len(word_tokenizer.word_index)+1
EMBED_DIM = 100
X_train, X_test, y_train, y_test = train_test_split(X_padded, y_padded, test_size=0.2, random_state=42)
y_train_exp = y_train[..., None]; y_test_exp = y_test[..., None]

input_layer = Input(shape=(MAX_LEN,))
embedding = Embedding(VOCAB_SIZE, EMBED_DIM, input_length=MAX_LEN, mask_zero=True)(input_layer)
bilstm = Bidirectional(LSTM(64, return_sequences=True))(embedding)
output = TimeDistributed(Dense(len(tag2idx), activation='softmax'))(bilstm)
ner_model = Model(input_layer, output)
ner_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
ner_model.fit(X_train, y_train_exp, validation_data=(X_test, y_test_exp), epochs=5, batch_size=16)

# Save
ner_model.save('/content/drive/MyDrive/PENS/ner_bilstm_model.keras')
joblib.dump(word_tokenizer, '/content/drive/MyDrive/PENS/ner_word_tokenizer.pkl')
joblib.dump(tag2idx, '/content/drive/MyDrive/PENS/ner_tag2idx.pkl')


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
150/150 ━━━━━━━━━━━━━━━━━━━━ 44s 131ms/step - accuracy: 0.9795 - loss: 0.2163 - val_accuracy: 1.0000 - val_loss: 1.0135e-04
Epoch 2/5
150/150 ━━━━━━━━━━━━━━━━━━━━ 17s 115ms/step - accuracy: 1.0000 - loss: 2.4197e-04 - val_accuracy: 1.0000 - val_loss: 6.0891e-05
Epoch 3/5
150/150 ━━━━━━━━━━━━━━━━━━━━ 7s 26ms/step - accuracy: 1.0000 - loss: 1.3564e-04 - val_accuracy: 1.0000 - val_loss: 4.1098e-05
Epoch 4/5
150/150 ━━━━━━━━━━━━━━━━━━━━ 4s 28ms/step - accuracy: 1.0000 - loss: 5.3449e-05 - val_accuracy: 1.0000 - val_loss: 2.8254e-05
Epoch 5/5
150/150 ━━━━━━━━━━━━━━━━━━━━ 5s 30ms/step - accuracy: 1.0000 - loss: 4.4609e-05 - val_accuracy: 1.0000 - val_loss: 2.1303e-05


['/content/drive/MyDrive/PENS/ner_tag2idx.pkl']

Step 9 – Extractive Summarization

In [ ]:
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer

def extractive_summary(text, num_sentences=3):
    sentences = sent_tokenize(text)
    if len(sentences)<=num_sentences:
        return text
    tfidf = TfidfVectorizer(stop_words='english')
    scores = tfidf.fit_transform(sentences).sum(axis=1).A1
    top_idx = np.argsort(scores)[-num_sentences:]
    return ' '.join([sentences[i] for i in sorted(top_idx)])


In [ ]:
!pip install streamlit pyngrok nltk


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 72.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 113.8 MB/s eta 0:00:00


In [ ]:
%%writefile app.py
import streamlit as st
import joblib
import numpy as np
import re
import nltk

from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.tokenize import sent_tokenize

# Download required tokenizer
nltk.download('punkt')

# --------------------
# Helpers
# --------------------
def clean_text(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    return text.lower().strip()

def extractive_summary(text, n=3):
    sentences = sent_tokenize(text)
    return " ".join(sentences[:n])

# --------------------
# Load Models (CACHED)
# --------------------
@st.cache_resource
def load_all_models():
    clf = joblib.load('/content/drive/MyDrive/PENS/logreg_model.pkl')
    tfidf = joblib.load('/content/drive/MyDrive/PENS/tfidf.pkl')
    label_encoder = joblib.load('/content/drive/MyDrive/PENS/label_encoder.pkl')

    dl_model = load_model('/content/drive/MyDrive/PENS/lstm_w2v_model.h5')
    dl_tokenizer = joblib.load('/content/drive/MyDrive/PENS/dl_tokenizer.pkl')

    ner_model = load_model('/content/drive/MyDrive/PENS/ner_bilstm_model.keras')
    ner_tokenizer = joblib.load('/content/drive/MyDrive/PENS/ner_word_tokenizer.pkl')
    tag2idx = joblib.load('/content/drive/MyDrive/PENS/ner_tag2idx.pkl')
    idx2tag = {v: k for k, v in tag2idx.items()}

    return clf, tfidf, label_encoder, ner_model, ner_tokenizer, idx2tag

clf, tfidf, label_encoder, ner_model, ner_tokenizer, idx2tag = load_all_models()

MAX_LEN = 100

# --------------------
# UI
# --------------------
st.title("📰 Multi-Task News Intelligence System (PENS Project)")

task = st.selectbox(
    "Select Task",
    ["Classification", "NER", "Summarization"]
)

text = st.text_area("Paste News Article Here")

if st.button("Run"):
    if not text.strip():
        st.warning("Please enter some text.")
    else:
        if task == "Classification":
            clean = clean_text(text)
            vec = tfidf.transform([clean])
            pred = clf.predict(vec)[0]
            label = label_encoder.inverse_transform([pred])[0]
            st.success(f"Predicted Category: {label}")

        elif task == "NER":
            tokens = text.lower().split()
            seq = ner_tokenizer.texts_to_sequences([tokens])
            padded = pad_sequences(seq, maxlen=MAX_LEN, padding='post')

            preds = ner_model.predict(padded, verbose=0)[0]
            entities = [
                (tokens[i], idx2tag[np.argmax(preds[i])])
                for i in range(len(tokens))
            ]
            st.write(entities)

        elif task == "Summarization":
            summary = extractive_summary(text)
            st.success(summary)



Writing app.py


In [ ]:
!pip install pyngrok

from pyngrok import ngrok

# Set your ngrok authtoken
ngrok.set_auth_token("33W9IXceAroYV3mge5rdq67S1Mm_6yw6afTVymn9wUQ3pJsM3")

In [ ]:
import os

# Kill any old tunnels
ngrok.kill()

# Start Streamlit in the background
get_ipython().system_raw("streamlit run app.py &")

# Start ngrok tunnel via HTTP
public_url = ngrok.connect(8501)
print("🔗 Your Streamlit app is live here:")
print(public_url)


🔗 Your Streamlit app is live here:
NgrokTunnel: "https://nonpsychopathic-overbrilliant-aaden.ngrok-free.dev" -> "http://localhost:8501"


In [ ]:
print(label_encoder.classes_)

['autos' 'entertainment' 'europe' 'finance' 'foodanddrink' 'health' 'kids'
 'lifestyle' 'movies' 'music' 'news' 'sports' 'travel' 'tv' 'video'
 'weather']
